In [6]:
## Stacked Autoencoders with MNIST ##
## url : http://deeplearning.net/tutorial/SdA.html ##
import numpy as np
import theano
import theano.tensor as T

from logistic import load_data,LogisticRegression
from mlp import HiddenLayer
from DA_module import DenoisingAutoencoder

from theano.tensor.shared_randomstreams import RandomStreams

In [8]:
class StackedAutoencoder(object):
    def __init__(self,rng,hidden_units,n_in=784,n_out=10,corruption_levels=[0.1,0.2,0.3]):
        self.num_layers = len[hidden_units]
        self.sigmoid_layers = []
        self.dA_layers = []
        self.params = []
        theano.rng = RandomStreams(rng.randint(2**30))
        # input data : x 
        self.x = T.matrix('x')
        # output data : y
        self.y = T.ivector('y')
        
        # setup the layers : input, size_of_input
        for i in xrange(self.num_layers):
            # if layer 0
            if i == 0:
                # layer 0 : takes n_in inputs
                layer_input_size = n_in
                layer_input = self.x
            else:
                layer_input_size = self.hidden_units[i-1]
                # previous layer's output
                # as the list of layers is built step by step
                #  the last layer in the list is the previous layer
                layer_input = sigmoid_layers[-1].output
            #construct a sigmoid layer    
            sigmoid_layer = HiddenLayer(rng=rng,
                                        input=layer_input,
                                        n_in=layer_input_size,
                                        n_out=self.hidden_units[i],
                                        activation=T.nnet.sigmoid)
            # add constructed sigmoid layer to list
            self.sigmoid_layers.append(sigmoid_layer)
            # append params of this layer to params collection of sDA
            self.params.extend(sigmoid_layer.params)
            # setup the corresponding autoencoder layer
            da = DenoisingAutoencoder(input=layer_input, rng=rng, num_v = layer_iput_size,
                                      num_h = self.hidden_units[i],theano_rng=theano_rng,
                                      w=sigmoid_layer.w,bh=sigmoid_layer.b)# param sharing
            self.dA_layers.append(da)
            
        # setupt the last logistic layer
        self.output_layer = LogisticRegression(input=self.sigmoid_layers[-1].output,n_in=self.hidden_units[-1],
                                         n_out = n_out)
        # add params of logistic layer to global params
        self.params.extend(output_layer.params)
        # fine_tune cost
        self.fine_tune_cost = self.output_layer.neg_log_likelihood(self.y)
                    
                
                